# Chapter 3: Schema and Types

Necessary imports:

In [1]:
import enum
from datetime import date, datetime

from sqlalchemy import (Boolean, CheckConstraint, Column, Date, DateTime, Enum,
                        ForeignKey, Index, Integer, MetaData, Numeric, String,
                        Table, create_engine)

Create the metadata object:

In [2]:
metadata = MetaData()

Define the employee table:

In [3]:
employee = Table(
    "employee",
    metadata,
    Column("employee_id", Integer, primary_key=True),
    Column(
        "manager_id",
        Integer,
        ForeignKey("employee.employee_id"),
        nullable=True
    ),
    Column("name", String(31), nullable=False),
    Column("is_manager", Boolean, default=False),
    Column('hire_date', Date, default=date.today),
)

Define product types and the product table:

In [4]:
class ProductType(enum.Enum):
    PHONE = 0
    ACCESSORY = 1
    OTHER = 2

In [5]:
product = Table(
    "product",
    metadata,
    Column("product_id", Integer, primary_key=True),
    Column("product_name", String(255), index=True),
    Column(
        "unit_price",
        Numeric(12, 2),
        CheckConstraint("unit_price>0"),
    ),
    Column(
        "units_in_stock",
        Integer,
        CheckConstraint("units_in_stock>=0"),
        default=0,
    ),
    Column("type", Enum(ProductType), default=ProductType.OTHER),
)

Define the customer table:

In [6]:
customer = Table(
    "customer",
    metadata,
    Column("customer_id", Integer, primary_key=True),
    Column("first_name", String(31)),
    Column("last_name", String(31)),
    Column("address", String(255)),
    Column("email", String(127), unique=True, nullable=False),
)

Compound index for customer table:


In [7]:
Index("customer_full_name", customer.c.first_name, customer.c.last_name)

Index('customer_full_name', Column('first_name', String(length=31), table=<customer>), Column('last_name', String(length=31), table=<customer>))

Defining a constraint outside of the `Table()` constructor:

In [8]:
CheckConstraint(
    product.c.units_in_stock < 1_000_000,
    name="stock_upper_limit",
)

CheckConstraint(<sqlalchemy.sql.elements.BinaryExpression object at 0x7ff00cf9f3a0>, name='stock_upper_limit', table=Table('product', MetaData(), Column('product_id', Integer(), table=<product>, primary_key=True, nullable=False), Column('product_name', String(length=255), table=<product>), Column('unit_price', Numeric(precision=12, scale=2), CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7ff00cf9f460>), table=<product>), Column('units_in_stock', Integer(), CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7ff00cf9f430>), table=<product>, default=ScalarElementColumnDefault(0)), Column('type', Enum('PHONE', 'ACCESSORY', 'OTHER', name='producttype'), table=<product>, default=ScalarElementColumnDefault(<ProductType.OTHER: 2>)), schema=None))

Define the order table:

In [9]:
order = Table(
    "order",
    metadata,
    Column("order_id", Integer, primary_key=True),
    Column(
        "customer_id",
        Integer,
        ForeignKey("customer.customer_id"),
        nullable=False,
    ),
    Column(
        "employee_id",
        Integer,
        ForeignKey("employee.employee_id"),
        nullable=True,
    ),
    Column("order_datetime", DateTime, default=datetime.now),
    Column("is_shipped", Boolean, default=False),
)

Define the order detail associative table:

In [10]:
order_detail = Table(
    "order_detail",
    metadata,
    Column(
        "order_id",
        Integer,
        ForeignKey("order.order_id"),
        primary_key=True,
    ),
    Column(
        "product_id",
        Integer,
        ForeignKey("product.product_id"),
        primary_key=True,
    ),
    Column("quantity", Integer,  default=1),
    CheckConstraint("quantity>0", name="num_of_ordered_item_must_be_positive"),
)

Enter your database URL (we'll be using SQLite3 with files):

In [11]:
DATABASE_URL = "sqlite+pysqlite:///store.db"

Create the `Engine` object:

In [12]:
engine = create_engine(
    DATABASE_URL,
    echo=True,
)

Create the tables:

In [13]:
metadata.create_all(engine)

2024-03-19 16:03:51,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-19 16:03:51,948 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-19 16:03:51,948 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 16:03:51,949 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-19 16:03:51,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 16:03:51,951 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-19 16:03:51,951 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 16:03:51,952 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-19 16:03:51,952 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 16:03:51,953 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-19 16:03:51,954 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 16:03:51,955 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customer")
2024-03-19 16:03:51,955 INFO sqlalchemy.engine.Engine [raw sql

Verify if the tables are created correctly:

```bash
$ sqlite3 store.db 
SQLite version 3.37.2 2022-01-06 13:25:41
Enter ".help" for usage hints.

# This command will display the SQL CREATE statements used to define each table in the database, including any indexes, triggers, or views.
sqlite> .schema

# When you're done:
sqlite> .exit
```